In [1]:
import sys

In [2]:
sys.path.append('../rover_based_aggregation/')

In [3]:
import utils.word_transition_network as wtn_module
from utils.rover import RoverVotingScheme
from utils.word_transition_network import *
from collections import Counter

In [4]:
import gensim

In [5]:
import yt.wrapper as yt
yt.config.set_proxy("hahn")

In [6]:
data_table = yt.read_table(
    "//home/voice/edvls/tickets/VA-442_ideal_testsets/assistant_ideal_annotations_2019-02-16__2019-02-25"
)
data_table = list(data_table)

In [7]:
AggregationResult = collections.namedtuple('AggregationResult', 'text confidence cost')

In [8]:
def aggregate_prod(raw_data):
    """
       aggregation from prod
    """
    cost = 2
    while cost < 5:
        cost += 1
        answers = [(x["text"], x["speech"]) for x in raw_data[:cost]]
        answers = Counter(answers)
        if answers.most_common(1)[0][1] >= 3:
            break

    texts = Counter()
    speechs = Counter()
    for text, speech in [(x["text"], x["speech"]) for x in raw_data[:cost]]:
        if speech != "BAD" and text:
            text = text.lower().replace('ё', 'е')
        else:
            text = ""
        speechs.update([speech])
        texts.update([text])
    
    
    text, text_rate = max(texts.items(), key=lambda x: (x[1], x[0] != ""))
    if text != "" and text_rate >= 2:
        conf = text_rate * 1.0 / sum(texts.values())
    else:
        text = None
        conf = 0
    common = speechs.most_common(2)
    speech, speech_rate = common[0]
    if speech == "BAD" and len(common) >= 2 and common[1][1] == speech_rate:
        speech = common[1][0]

    # conf = text_rate / sum(texts.values())
    return AggregationResult(text, conf, cost)

In [9]:
def evaluate_metrics(data, field, algorithm, treshhold=0, cluster_refernces=None, print_=True):
    errors = 0
    total_length = 0
    aggregated = 0
    total_items = 0
    correct = 0
    cost = 0
    for row in data:
        if row["mark"] != "TEST":
            continue
        total_items += 1
        hyp = algorithm(sorted(row[field], key=lambda x: x["submit_ts"]))
        cost += hyp.cost
        if (hyp.text is None) or (hyp.confidence < treshhold):
            continue
        hyp = hyp.text
        aggregated += 1
        _, e, l = calculate_wer(row["text"], hyp, cluster_refernces)
        errors += e
        if e == 0:
            correct += 1
        total_length += l

    accuracy = correct / aggregated
    wer = errors / total_length
    aggregated_part = aggregated / total_items
    cost = cost / total_items
    if print_:
        print("Aggregated: {:.4%}\nWER: {:.4%}\nAccuracy: {:.4%}\nMean overlap: {:.4}".format(
            aggregated_part, wer, accuracy, cost
        ))
    return aggregated_part, wer, accuracy, cost

In [10]:
for i in data_table[0].keys():
    print(i)

text
linguists_comment
yang_assignments_repeat_1
audio
toloka_assignments_repeat_10
toloka_assignments_repeat_1
toloka_assignments_repeat_11_selected_workers_with_pitch
toloka_speech
toloka_assignments_repeat_3_with_pitch
mark
_other
linguists_sugested_text
toloka_assignments_repeat_7_with_chorus_and_pitch
linguists_worker_id
toloka_assignments
speech
url
toloka_assignments_repeat_2_with_pitch
toloka_assignments_repeat_6_with_chorus
toloka_assignments_repeat_4_with_bend
date
mds_key
check_in_yang_results
raw_text_linguists
toloka_assignments_repeat_9_selected_workers_with_chorus_and_pitch
toloka_assignments_repeat_8_selected_workers_with_chorus_and_pitch
toloka_assignments_repeat_5_with_chorus_and_pitch
toloka_number_of_speakers
number_of_speakers
toloka_text


In [11]:
evaluate_metrics(data_table, 'toloka_assignments_repeat_11_selected_workers_with_pitch', aggregate_prod)

Aggregated: 67.9862%
WER: 7.3166%
Accuracy: 78.9030%
Mean overlap: 3.833


(0.6798623063683304,
 0.07316627034936894,
 0.7890295358649789,
 3.8330464716006887)

In [12]:
aggregate_prod(data_table[2]['toloka_assignments_repeat_11_selected_workers_with_pitch'])

AggregationResult(text='алиса что такое форма три справка', confidence=1.0, cost=3)

In [1]:
fasttext_model=gensim.models.FastText.load_fasttext_format('fasttext_output.bin')

NameError: name 'gensim' is not defined

In [14]:
def aggregate_centroid(raw_data):
    annot_topk=[a['text'] for a in raw_data]
    centroid_scores=[]
    if len(set(annot_topk))==1:
        text=annot_topk[0]
    else:
        lengths=[len(set(elem for elem in annot_topk if elem!=cur)) for cur in annot_topk]
        for annot,length in zip(annot_topk,lengths):
            centroid_scores.append(np.sum([fasttext_model.wv.wmdistance([annot],[other_annot]) for other_annot in annot_topk])/length)
        text=annot_topk[np.argmin(centroid_scores)]
    conf=1
    if text=='':
        text=None
        conf=0
    return AggregationResult(text,conf,len(annot_topk))

In [15]:
evaluate_metrics(data_table, 'toloka_assignments_repeat_11_selected_workers_with_pitch', aggregate_centroid)

Aggregated: 75.7889%
WER: 9.1768%
Accuracy: 74.7918%
Mean overlap: 10.0


(0.7578886976477338, 0.09176768052353684, 0.7479182437547313, 10.0)

In [16]:
import json
from collections import Counter
import numpy as np

In [17]:
assistant_ideal=[]
with open('assistant_ideal.jsonl',encoding='utf8') as f:
    for line in f:
        assistant_ideal.append(json.loads(line.strip()))

In [18]:
quasar_ideal=[]
with open('quasar_ideal.jsonl',encoding='utf8') as f:
    for line in f:
        quasar_ideal.append(json.loads(line.strip()))

In [19]:
def upper_bound(logs):
    n=[]
    for sample in logs:
        true=sample['text']
        annot=[a['text'] for a in sample['annotations']]
        n.append(int(true in annot))
    return np.mean(n),np.sum(n)

In [20]:
def mv_quality(logs,topk=5):
    n=[]
    pre_df={'golden_ans':[],'mv_ans':[]}
    for sample in logs:
        true=sample['text']
        annot=[a['text'] for a in sample['annotations']]
        cnt=Counter(annot[:topk])#.most_common(1)[0]
        mv=max(cnt,key=lambda k:cnt[k])
        n.append(int(mv==true))
        pre_df['golden_ans'].append(true)
        pre_df['mv_ans'].append(mv)
    return np.mean(n),np.sum(n)

In [21]:
upper_bound(assistant_ideal)

(0.8958986043862147, 6291)

In [22]:
len(assistant_ideal)

7022

In [23]:
mean,sum_=mv_quality(assistant_ideal)

In [24]:
mean,sum_

(0.7569068641412703, 5315)

In [25]:
mv_quality(quasar_ideal)

(0.8175824175824176, 5952)

In [26]:
mv_quality(assistant_ideal,topk=50)

(0.7851039589860439, 5513)

In [27]:
mv_quality(quasar_ideal,topk=50)

(0.8368131868131868, 6092)

In [28]:
def production_method_quality(logs):
    n=[]
    for sample in logs:
        true=sample['text']
        annot=[a['text'] for a in sample['annotations']]
        mv=Counter(annot[:3]).most_common(1)[0]
        if mv[1]>1:
            n.append(int(mv[0]==true))
        else:
            mv=Counter(annot[:4]).most_common(1)[0]
            if mv[1]>1: # (2,2) is impossible
                n.append(int(mv[0]==true))
            else:
                mv=Counter(annot[:5]).most_common(1)[0]
                if mv[1]>1:
                    n.append(int(mv[0]==true))
                else:
                    n.append(int(annot[0]==true))
    return np.mean(n)

In [29]:
production_method_quality(assistant_ideal)

0.7529193961834235

In [30]:
production_method_quality(quasar_ideal)

0.8108516483516484

In [31]:
def centroid_quality(logs,topk=5):
    n=[]
    for sample in logs:
        true=sample['text']
        annot_topk=[a['text'] for a in sample['annotations']][:topk]
        centroid_scores=[]
        for annot in annot_topk:
            centroid_scores.append(np.mean([fasttext_model.wv.wmdistance([annot],[other_annot]) for other_annot in annot_topk]))
        n.append(int(annot_topk[np.argmin(centroid_scores)]==true))
    return np.mean(n)

In [32]:
centroid_quality(assistant_ideal,topk=50)

0.785246368555967

In [33]:
centroid_quality(quasar_ideal,topk=50)

0.8368131868131868

In [34]:
centroid_quality(assistant_ideal,topk=5)

0.7634577043577329

In [35]:
centroid_quality(quasar_ideal,topk=5)

0.817032967032967

In [36]:
centroid_quality(assistant_ideal,topk=3)

0.7314155511250356

In [37]:
centroid_quality(quasar_ideal,topk=3)

0.7953296703296703

In [38]:
def centroid_quality_median(logs):
    n=[]
    for sample in logs:
        true=sample['text']
        annot_top5=[a['text'] for a in sample['annotations']][:5]
        centroid_scores=[]
        for annot in annot_top5:
            centroid_scores.append(np.median([fasttext_model.wv.wmdistance([annot],[other_annot]) for other_annot in annot_top5]))
        n.append(int(annot_top5[np.argmin(centroid_scores)]==true))
    return np.mean(n)

In [39]:
centroid_quality_median(assistant_ideal)

0.753631444033039

In [40]:
centroid_quality_median(quasar_ideal)

0.8083791208791209